### **Khởi tạo Spark và Tải Dữ liệu**

**Các bước thực hiện:**

1.  **Dữ liệu đầu vào:** File `creditcardreal.csv` chứa thông tin giao dịch thẻ tín dụng.
2.  **Các bước tính toán:**
    *   Khởi tạo `SparkSession`, là điểm truy cập chính để sử dụng các chức năng của Spark.
    *   Sử dụng `spark.sparkContext.textFile()` để đọc dữ liệu từ file CSV vào một RDD. Mỗi dòng trong file sẽ trở thành một phần tử trong RDD.
    *   Tách dòng tiêu đề ra khỏi RDD chính để không ảnh hưởng đến quá trình tính toán.
    *   Chuyển đổi mỗi dòng dữ liệu thành một danh sách các giá trị bằng cách tách chuỗi tại dấu phẩy.
3.  **Kết quả trả về:** Một RDD chứa dữ liệu giao dịch đã được làm sạch sơ bộ (loại bỏ tiêu đề) và sẵn sàng cho các bước phân tích tiếp theo. Hiển thị 5 dòng đầu tiên và tổng số lượng giao dịch để kiểm tra.

In [ ]:
# Import SparkSession từ thư viện pyspark.sql
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
# SparkSession là điểm bắt đầu để làm việc với Spark.
# appName("CreditCardAnalysis") đặt tên cho ứng dụng.
# getOrCreate() sẽ lấy SparkSession hiện có hoặc tạo một cái mới nếu chưa có.
spark = SparkSession.builder.appName("CreditCardAnalysis").getOrCreate()

# Tải dữ liệu từ file creditcardreal.csv vào RDD
# spark.sparkContext.textFile() được dùng để đọc file văn bản và tạo RDD,
# mỗi dòng trong file sẽ là một phần tử của RDD.
file_path = "creditcardreal.csv"
lines_rdd = spark.sparkContext.textFile(file_path)

# Lấy dòng tiêu đề để loại bỏ khỏi dữ liệu chính
header = lines_rdd.first()

# Lọc bỏ dòng tiêu đề và xử lý dữ liệu
# Dùng filter() để loại bỏ dòng tiêu đề.
# Dùng map() để tách mỗi dòng thành các cột bằng dấu phẩy (,).
data_rdd = lines_rdd.filter(lambda row: row != header).map(lambda line: line.split(','))

# Cache RDD để tăng tốc độ cho các lần truy cập sau
data_rdd.cache()

# In ra 5 dòng đầu tiên để kiểm tra xem dữ liệu đã được đọc đúng chưa
print("5 dòng dữ liệu đầu tiên:")
for row in data_rdd.take(5):
    print(row)

# Đếm và in ra tổng số dòng dữ liệu đã được tải (không tính tiêu đề)
print(f"\nTổng số dòng dữ liệu (không tính tiêu đề): {data_rdd.count()}")

### **Giải thích kết quả**

Kết quả trên cho thấy:
- **SparkSession đã được khởi tạo thành công**.
- **Dữ liệu đã được tải từ file `creditcardreal.csv` vào một RDD**.
- **5 dòng đầu tiên của dữ liệu** được hiển thị, mỗi dòng là một danh sách (list) các chuỗi (string), tương ứng với các cột trong file CSV. Điều này xác nhận rằng việc đọc và tách dữ liệu đã thành công.
- **Tổng số lượng giao dịch** trong tập dữ liệu (không bao gồm dòng tiêu đề) đã được đếm và hiển thị. Con số này cho chúng ta cái nhìn tổng quan về quy mô của dữ liệu.
- **RDD đã được cache()** để các thao tác tính toán sau này trên cùng một RDD sẽ nhanh hơn.

Dữ liệu giờ đã sẵn sàng để thực hiện các bước phân tích chi tiết hơn ở các câu tiếp theo.

### **Yêu cầu 1: Phân tích giá trị giao dịch (Amount) theo từng loại (Class)**

**Các bước thực hiện:**

1.  **Dữ liệu đầu vào:** RDD `data_rdd` đã được xử lý ở bước trước.
2.  **Các bước tính toán:**
    *   Tạo một RDD mới (`class_amount_rdd`) chứa các cặp key-value, với `key` là `Class` (cột cuối cùng) và `value` là `Amount` (cột kế cuối).
    *   Chuyển đổi giá trị `Amount` từ kiểu chuỗi sang kiểu số thực (float) để tính toán. Làm sạch giá trị `Class` để loại bỏ các ký tự không mong muốn (ví dụ: dấu ngoặc kép).
    *   Sử dụng `combineByKey` để tính toán đồng thời nhiều giá trị thống kê cho mỗi `Class` một cách hiệu quả. Phương thức này sẽ tính tổng (sum), số lượng (count), giá trị nhỏ nhất (min), và giá trị lớn nhất (max) của `Amount` cho mỗi `Class`.
    *   Từ kết quả tổng và số lượng, tính giá trị trung bình (average).
3.  **Kết quả trả về:** Các giá trị thống kê mô tả (tổng, trung bình, min, max) của `Amount` cho mỗi loại giao dịch (hợp lệ và gian lận).

In [ ]:
# Tạo RDD key-value với (Class, Amount)
# Cột cuối cùng (index -1) là 'Class', cột kế cuối (index -2) là 'Amount'
# Loại bỏ dấu ngoặc kép khỏi giá trị Class và chuyển Amount sang float
class_amount_rdd = data_rdd.map(lambda row: (row[-1].strip('"'), float(row[-2])))

# Sử dụng combineByKey để tính toán các giá trị thống kê
# (sum, count, min, max)
sum_count_rdd = class_amount_rdd.combineByKey(
    (lambda value: (value, 1, value, value)),  # createCombiner: (sum, count, min, max)
    (lambda x, value: (x[0] + value, x[1] + 1, min(x[2], value), max(x[3], value))),  # mergeValue
    (lambda x, y: (x[0] + y[0], x[1] + y[1], min(x[2], y[2]), max(x[3], y[3])))  # mergeCombiners
)

# Tính giá trị trung bình từ tổng và số lượng
stats_rdd = sum_count_rdd.mapValues(lambda x: {
    'sum': x[0],
    'count': x[1],
    'min': x[2],
    'max': x[3],
    'mean': x[0] / x[1]
})

# Thu thập và in kết quả
results = stats_rdd.collect()

print("Phân tích giá trị giao dịch theo từng loại:")
for result in results:
    class_type = 'Hợp lệ' if result[0] == '0' else 'Gian lận'
    print(f"\n--- Loại giao dịch: {class_type} (Class {result[0]}) ---")
    print(f"Tổng giá trị giao dịch: {result[1]['sum']:.2f}")
    print(f"Số lượng giao dịch: {result[1]['count']}")
    print(f"Giá trị giao dịch trung bình: {result[1]['mean']:.2f}")
    print(f"Giá trị giao dịch nhỏ nhất: {result[1]['min']:.2f}")
    print(f"Giá trị giao dịch lớn nhất: {result[1]['max']:.2f}")

### **Giải thích kết quả Yêu cầu 1**

Kết quả phân tích cho thấy sự khác biệt rõ rệt về giá trị giao dịch giữa hai loại:

- **Giao dịch Hợp lệ (Class 0):**
  - Có tổng giá trị và số lượng giao dịch vượt trội, chiếm phần lớn trong tập dữ liệu.
  - Giá trị giao dịch trung bình thấp hơn đáng kể so với các giao dịch gian lận.
  - Dải giá trị giao dịch rất rộng, từ 0 cho đến một giá trị rất lớn, phản ánh sự đa dạng trong hành vi mua sắm thông thường.

- **Giao dịch Gian lận (Class 1):**
  - Mặc dù số lượng giao dịch ít, nhưng giá trị giao dịch trung bình lại cao hơn hẳn. Điều này cho thấy kẻ gian thường nhắm đến việc rút số tiền lớn trong mỗi lần giao dịch.
  - Giá trị giao dịch lớn nhất của nhóm này thấp hơn so với nhóm hợp lệ, có thể do các biện pháp an ninh đã chặn các giao dịch gian lận có giá trị quá lớn.

Những thông tin này rất hữu ích cho việc xây dựng các mô hình phát hiện gian lận, vì `Amount` là một đặc trưng quan trọng để phân biệt giữa hai loại giao dịch.

### **Yêu cầu 2: Giá trị giao dịch trung bình của mỗi nhóm là bao nhiêu?**

**Các bước thực hiện:**

1.  **Dữ liệu đầu vào:** RDD `class_amount_rdd` đã được tạo ở Yêu cầu 1, chứa các cặp (`Class`, `Amount`).
2.  **Các bước tính toán:**
    *   Sử dụng `combineByKey` để tính tổng giá trị (`sum`) và tổng số lượng (`count`) cho mỗi `Class`.
        *   `createCombiner`: Khởi tạo một tuple `(value, 1)` cho mỗi key mới, tương ứng với `(sum, count)`.
        *   `mergeValue`: Cộng dồn `value` vào `sum` và tăng `count` lên 1 cho mỗi phần tử mới có cùng key.
        *   `mergeCombiners`: Kết hợp các kết quả từ các partition khác nhau.
    *   Sử dụng `mapValues` để tính giá trị trung bình bằng cách lấy `sum` chia cho `count`.
3.  **Kết quả trả về:** Giá trị giao dịch trung bình cho mỗi loại (hợp lệ và gian lận).

In [ ]:
# Tính tổng và số lượng cho mỗi Class
sum_count_avg_rdd = class_amount_rdd.combineByKey(
    (lambda value: (value, 1)),  # createCombiner: (sum, count)
    (lambda x, value: (x[0] + value, x[1] + 1)),  # mergeValue
    (lambda x, y: (x[0] + y[0], x[1] + y[1]))  # mergeCombiners
)

# Tính giá trị trung bình
average_rdd = sum_count_avg_rdd.mapValues(lambda x: x[0] / x[1])

# Thu thập và in kết quả
avg_results = average_rdd.collect()

print("Giá trị giao dịch trung bình của mỗi nhóm:")
for result in avg_results:
    class_type = 'Hợp lệ' if result[0] == '0' else 'Gian lận'
    print(f"- {class_type} (Class {result[0]}): {result[1]:.2f}")

### **Giải thích kết quả Yêu cầu 2**

Kết quả này tái khẳng định những gì đã quan sát ở Yêu cầu 1:

- **Giá trị trung bình của giao dịch hợp lệ** là một con số tương đối nhỏ, phản ánh các chi tiêu hàng ngày của người dùng.
- **Giá trị trung bình của giao dịch gian lận** cao hơn đáng kể. Điều này củng cố giả thuyết rằng các hoạt động gian lận thường có xu hướng thực hiện các giao dịch với số tiền lớn để tối đa hóa lợi ích bất chính trước khi bị phát hiện.

### **Yêu cầu 3: Giao dịch có giá trị Amount > 1000**

**Các bước thực hiện:**

1.  **Dữ liệu đầu vào:** RDD `data_rdd` chứa toàn bộ dữ liệu giao dịch.
2.  **Các bước tính toán:**
    *   Sử dụng `filter()` để lọc ra những giao dịch có giá trị `Amount` (cột kế cuối) lớn hơn 1000.
    *   Tạo một RDD mới từ dữ liệu đã lọc, chỉ chứa cột `Class`.
    *   Sử dụng `map()` để tạo các cặp key-value `(Class, 1)`.
    *   Sử dụng `reduceByKey()` để đếm số lượng giao dịch cho mỗi `Class`.
3.  **Kết quả trả về:** Số lượng giao dịch gian lận và hợp lệ trong số các giao dịch có giá trị `Amount` lớn hơn 1000.

In [ ]:
# Lọc các giao dịch có Amount > 1000
large_transactions_rdd = data_rdd.filter(lambda row: float(row[-2]) > 1000)

# Đếm số lượng giao dịch theo Class trong tập dữ liệu đã lọc
count_by_class_rdd = large_transactions_rdd.map(lambda row: (row[-1].strip('"'), 1)) \
                                          .reduceByKey(lambda a, b: a + b)

# Thu thập và in kết quả
count_results = count_by_class_rdd.collect()

print("Số lượng giao dịch có Amount > 1000:")
for result in count_results:
    class_type = 'Hợp lệ' if result[0] == '0' else 'Gian lận'
    print(f"- {class_type} (Class {result[0]}): {result[1]} giao dịch")

### **Giải thích kết quả Yêu cầu 3**

Kết quả cho thấy trong số các giao dịch có giá trị lớn (trên 1000), có cả giao dịch hợp lệ và gian lận.

- **Số lượng giao dịch hợp lệ** vẫn chiếm đa số, điều này là tự nhiên vì khách hàng có thể thực hiện các giao dịch lớn cho các mặt hàng đắt tiền như đồ điện tử, du lịch, v.v.
- **Sự tồn tại của các giao dịch gian lận** trong nhóm này nhấn mạnh tầm quan trọng của việc giám sát các giao dịch có giá trị cao, vì đây là mục tiêu hấp dẫn của tội phạm thẻ tín dụng. Mặc dù số lượng ít, nhưng thiệt hại từ mỗi giao dịch này có thể rất lớn.

### **Yêu cầu 4: Phân tích số lượng giao dịch theo giờ và tìm giờ cao điểm**

**Các bước thực hiện:**

1.  **Dữ liệu đầu vào:** RDD `data_rdd` chứa toàn bộ dữ liệu giao dịch.
2.  **Các bước tính toán:**
    *   Trích xuất cột `Time` (cột đầu tiên) từ RDD. Cột này biểu thị số giây kể từ giao dịch đầu tiên.
    *   Chuyển đổi số giây thành giờ trong ngày. Một ngày có 86400 giây. Công thức: `hour = (time_in_seconds % 86400) // 3600`.
    *   Tạo RDD key-value với `key` là giờ trong ngày (0-23) và `value` là `1`.
    *   Sử dụng `reduceByKey` để đếm tổng số giao dịch cho mỗi giờ.
    *   Lưu kết quả vào cache (`transactions_by_hour_rdd.cache()`) vì nó sẽ được tái sử dụng cho Yêu cầu 5.
    *   Tìm ra giờ có số lượng giao dịch cao nhất bằng cách sử dụng `max()` với một hàm `key` để so sánh giá trị count.
3.  **Kết quả trả về:** Giờ có nhiều giao dịch nhất và số lượng giao dịch tương ứng.

In [ ]:
# Trích xuất cột Time và chuyển đổi sang giờ trong ngày
hour_rdd = data_rdd.map(lambda row: (int(float(row[0])) % 86400 // 3600, 1))

# Đếm số lượng giao dịch theo giờ
transactions_by_hour_rdd = hour_rdd.reduceByKey(lambda a, b: a + b)

# Cache RDD này vì sẽ dùng lại ở Yêu cầu 5
transactions_by_hour_rdd.cache()

# Thu thập và sắp xếp kết quả để xem (tùy chọn)
sorted_hours = transactions_by_hour_rdd.sortBy(lambda x: x[0]).collect()

# Tìm giờ có nhiều giao dịch nhất
peak_hour = transactions_by_hour_rdd.max(key=lambda x: x[1])

print("Số lượng giao dịch theo từng giờ trong ngày:")
for hour, count in sorted_hours:
    print(f"- Giờ {hour}: {count} giao dịch")

print(f"\n=> Giờ có nhiều giao dịch nhất là giờ {peak_hour[0]} với {peak_hour[1]} giao dịch.")

### **Giải thích kết quả Yêu cầu 4**

Kết quả phân tích cho thấy sự phân bố số lượng giao dịch không đồng đều trong ngày.

- **Giờ cao điểm** được xác định là giờ có số lượng giao dịch cao nhất. Điều này thường tương ứng với các khung giờ mà người tiêu dùng hoạt động mua sắm nhiều nhất, ví dụ như giờ nghỉ trưa hoặc buổi tối sau giờ làm việc.
- **Các giờ có ít giao dịch nhất** thường rơi vào ban đêm hoặc sáng sớm, khi hầu hết mọi người đang ngủ.

Thông tin này có thể giúp các tổ chức tài chính dự đoán và phân bổ tài nguyên hệ thống tốt hơn để xử lý lượng giao dịch tăng đột biến vào các giờ cao điểm.

### **Yêu cầu 5: Dùng biểu đồ đường (Line Chart) để thể hiện xu hướng biến động**

**Các bước thực hiện:**

1.  **Dữ liệu đầu vào:** RDD `transactions_by_hour_rdd` đã được tính toán và cache ở Yêu cầu 4. RDD này chứa các cặp (`hour`, `count`).
2.  **Các bước tính toán:**
    *   Thu thập dữ liệu từ RDD về driver node bằng `collect()`.
    *   Sắp xếp dữ liệu theo giờ để đảm bảo biểu đồ được vẽ đúng thứ tự.
    *   Tách dữ liệu thành hai danh sách riêng biệt: `hours` cho trục hoành (X) và `counts` cho trục tung (Y).
    *   Sử dụng thư viện `matplotlib.pyplot` để vẽ biểu đồ đường.
    *   Thiết lập các thuộc tính cho biểu đồ như tiêu đề, nhãn cho các trục X và Y bằng tiếng Việt để dễ hiểu.
3.  **Kết quả trả về:** Một biểu đồ đường hiển thị trực quan xu hướng biến động của số lượng giao dịch qua các giờ trong ngày.

In [ ]:
# Import thư viện matplotlib
import matplotlib.pyplot as plt

# Lấy dữ liệu đã được sắp xếp từ bước trước
# transactions_by_hour_rdd đã được cache lại nên không cần tính toán lại
sorted_transactions = transactions_by_hour_rdd.sortBy(lambda x: x[0]).collect()

# Tách giờ và số lượng ra hai danh sách riêng biệt
hours = [item[0] for item in sorted_transactions]
counts = [item[1] for item in sorted_transactions]

# Vẽ biểu đồ đường
plt.figure(figsize=(12, 6))
plt.plot(hours, counts, marker='o', linestyle='-')

# Thêm tiêu đề và nhãn cho các trục
plt.title('Xu hướng biến động của số lượng giao dịch theo giờ')
plt.xlabel('Giờ trong ngày')
plt.ylabel('Số lượng giao dịch')
plt.xticks(range(0, 24)) # Đảm bảo hiển thị tất cả các giờ trên trục X
plt.grid(True)

# Hiển thị biểu đồ
plt.show()

### **Giải thích kết quả Yêu cầu 5**

Biểu đồ đường trực quan hóa một cách rõ ràng dữ liệu đã được tính toán ở Yêu cầu 4:

- **Trục hoành (X)** biểu thị 24 giờ trong một ngày.
- **Trục tung (Y)** biểu thị tổng số lượng giao dịch tương ứng với mỗi giờ.

Từ biểu đồ, ta có thể dễ dàng nhận thấy:
- **Xu hướng chung:** Số lượng giao dịch bắt đầu ở mức thấp vào đầu ngày (khoảng từ 0 đến 6 giờ sáng), tăng dần và đạt đỉnh vào các khung giờ nhất định, sau đó giảm dần về đêm.
- **Các điểm đỉnh và đáy:** Có thể xác định trực quan các giờ cao điểm (đỉnh của đường cong) và các giờ yên tĩnh nhất (đáy của đường cong). Điều này cung cấp một cái nhìn tổng thể và nhanh chóng về hành vi giao dịch của khách hàng trong một ngày.